# Libraries

In [ ]:
#________________ Libraries ________________
import pandas as pd
import collections
import numpy as np
import re
import random
import scipy as sc
import matplotlib.pyplot as plt
import statsmodels.api as sm
from math import radians, cos, sin, asin, sqrt
from sklearn.cluster import KMeans
from matplotlib.patches import Ellipse
!pip install haversine
from haversine import haversine, Unit, inverse_haversine, Direction

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data

In [ ]:
#________________ Import Data ________________
data = pd.read_csv('ProjectData.csv')
data = data.reset_index()
data.head()

,index,Call ID,Time,Lat,Lon,Safe to fly,Heli avail,CancelDelay,Scene time,Hosp lat,Hosp Lon,Hosp time
0,0,0,0.698273,43.606190,75.110334,1,1,0.0,0.376066,43.1009,75.2327,0.741709
1,1,1,0.790623,43.086142,77.470693,1,1,0.0,0.494475,43.1566,77.6088,0.276576
2,2,2,3.445114,42.781114,73.757133,1,1,0.0,0.223929,42.6526,73.7562,0.265198
3,3,3,3.742029,42.523381,76.452085,1,1,0.0,0.232449,42.4440,76.5019,0.494395
4,4,4,6.399294,43.204620,75.936761,1,1,0.0,0.447068,43.0481,76.1474,1.189299


# Final Variables

In [ ]:
#____________________ Final Variables ________________
speed = 160
max_distance = 180
call_rates = { 
            1 : 0.723, 2 : 0.618, 3 : 0.374, 4 : 0.319, 
            5 : 0.266, 6 : 0.277, 7 : 0.489, 8 : 0.777,
            9 : 1.70, 10 : 2.75, 11 : 3.19, 12 : 3.72,
            13 : 3.92, 14 : 3.67, 15 : 3.81,16 : 3.68, 
            17 : 3.56, 18 : 3.20,  19 : 3.05, 20 : 2.28,
            21 : 1.40, 22 : 1.17, 23 : 0.881, 24 : 0.813
            }
hosp_names = np.array(['Sarye',
                       'Albany',
                       'Syracuse',
                       'Rochester',
                       'Elmira',
                       'Binghamton',
                       'Ithaca',
                       'Buffalo',
                       'Utica',
                       'Watertown'])
                      
hosp_loc = np.array([[41.979, 76.5155], 
                     [42.6526, 73.7562] , 
                     [43.0481, 76.1474] , 
                     [43.1566  , 77.6088] , 
                     [42.0898 , 76.8077] , 
                     [42.0987, 75.918 ], 
                     [42.444 , 76.5019] , 
                     [42.8864 , 78.8784] , 
                     [43.1009 , 75.2327] , 
                     [43.9748 , 75.9108]])

#Generate Lamba for CancelDelay
safe_df = data[data['Safe to fly'] == 1]
safe_df = safe_df[safe_df['Heli avail'] == 1]
cancel_time = np.array(safe_df['CancelDelay'])
nonzero_cancel_time = cancel_time[cancel_time > 0] 
n_observed = len(nonzero_cancel_time)
n_unobserved = len(cancel_time) - n_observed 
s = sum(nonzero_cancel_time)
lambCancelDelay = n_observed/(s + 0.5*n_unobserved)

# Helper Functions

In [ ]:
#____________________ Helper Functions ____________________

#Array of Random Helicopter Placements
def randomHeliAmounts(n):
  #More clumped up 
  if np.random.rand() < .5:
    length = len(hosp_with_bases)
    arr = np.zeros(length)
    count = 0
    upper = n-1
    for i in range(length):
      num = np.random.randint(0, upper+1)
      arr[i] = num
      upper = upper - num
      count += num

    arr[length-1] = n-count
    random.shuffle(arr)
  #More distubuted
  else:
    length = len(hosp_with_bases)
    arr = np.zeros(length) 
    for i in range(n):
      index = np.random.randint(0, length)
      arr[index] = arr[index] + 1
  return arr

#Haversine Distance 
def haversine_fun(lon1, lat1, lon2, lat2):
  #lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
  #dlon = lon2 - lon1 
  #dlat = lat2 - lat1 
  #a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
  #c = 2 * asin(sqrt(a)) 
  #r = 6371 
  #return c * r
  return haversine([lat1,lon1],[lat2,lon2])

#Min distance from Base -> Scene
def minHD(lon_scene, lat_scene, heli_arr):
  min_HD_index = -1
  min_distance = 5000
  for i in range(len(hosp_with_bases)):
    lat_base = bases_loc[i][0]
    lon_base = bases_loc[i][1]
    temp_distance = haversine_fun(lon_scene, lat_scene, lon_base, lat_base)
    if(heli_arr[i] > 0 and temp_distance < min_distance and temp_distance <= 180):
      min_HD_index = i
      min_distance = temp_distance
  return min_HD_index, min_distance

#Min distance from Scene -> Hospital
def minHospital(lon_scene, lat_scene):
    major_name = hosp_names[0:4]
    major_loc = hosp_loc[0:4]
    minor_name = hosp_names[4:10]
    minor_loc = hosp_loc[4:10]
    
    major_distance = [0,0,0,0]
    minor_distance = [0,0,0,0,0,0]
    
    for i in range(len(major_loc)):
        major_distance[i] = haversine_fun(lon_scene, lat_scene, major_loc[i][1], major_loc[i][0])     
    for i in range(len(minor_loc)):
        minor_distance[i] = haversine_fun(lon_scene, lat_scene, minor_loc[i][1], minor_loc[i][0])
   
    min_minor = min(minor_distance)
    min_minor_index = minor_distance.index(min_minor)
    min_major = min(major_distance)
    min_major_index = major_distance.index(min_major)
    rand_bern = np.random.binomial(size = 1,n =1, p = 0.807)[0]
    if(min_major < min_minor ): #major trauma center is closest
        min_hospital_index = min_major_index
        min_distance = min_major
    elif(rand_bern == 1): #going to closest medical center 
        min_hospital_index = min_minor_index + 4 
        min_distance = min_minor
    else : #going to closest trauma center with p = 1-0.807
        min_hospital_index =  min_major_index 
        min_distance = min_major
    return min_hospital_index, min_distance

#Distance from Hospital -> Base
def hosptitalToBaseDist(base_index, hospital_index):
  lon_Hos = hosp_loc[hospital_index][1]
  lat_Hos = hosp_loc[hospital_index][0]
  lon_Base = bases_loc[base_index][1]
  lat_Base = bases_loc[base_index][0]
  return haversine_fun(lon_Hos, lat_Hos, lon_Base, lat_Base)

directions = [Direction.NORTH, Direction.EAST, Direction.SOUTH, Direction.WEST,
             Direction.NORTHEAST, Direction.NORTHWEST, Direction.SOUTHEAST, Direction.SOUTHWEST]

def jitter(scene_loc):
  jitter_rv = np.random.uniform()
  if jitter_rv <= 0.5:
      jitter = np.random.uniform()
      scene_loc = inverse_haversine(scene_loc, jitter, random.choice(directions))
  return(scene_loc)

def call_rate(rate):
  current_time = 0
  call_times = []
  while current_time <1 :
    inter_time = np.random.exponential(rate)
    call_times.append(inter_time)
    current_time += inter_time
  return np.sort(call_times)


# Core Simulation Functions

In [ ]:
#____________________ Core Simulation ____________

def simFunctionDays(numSim, amount_of_helis, hosp_with_bases, bases_loc):
  #Statstic Variables
  calls_dispacted = [] 
  responce_time = []
  time_to_care = []
  response_fraction = []

  #System Variables
  heli_arr_ORGINAL = randomHeliAmounts(amount_of_helis) 
  heli_arr = np.array(heli_arr_ORGINAL)
  overallTime = 0
  heli_not_returned_baseID = []
  heli_return_time = []

  #Number of days
  for d in range(0, numSim):
    #Number of hours in a day
    for i in range(1,25):      
      rate = call_rates[i]
      time_of_calls = call_rate(rate)
      number_of_calls = len(time_of_calls)
      #Number of calls this hour
      for j in range(number_of_calls):     
        current_case = data.sample() 
        overallTime = i + time_of_calls[j]/60 + (24*d)   
      
        #Helicopters returning back to base
        new_returned_baseID = []
        new_return_time = []
        for h in range(len(heli_return_time)):
          #Add Back
          if heli_return_time[h] <= overallTime:
            index = heli_not_returned_baseID[h]
            heli_arr[index] =  heli_arr[index] + 1
          #Remove from lst
          else:
            new_returned_baseID.append(heli_not_returned_baseID[h])
            new_return_time.append(heli_return_time[h])

        heli_not_returned_baseID =  new_returned_baseID
        heli_return_time = new_return_time

        safe_to_fly = sc.stats.bernoulli.rvs(.89925)
        #Safe to Fly?
        if(safe_to_fly == 1):       
          lon = current_case['Lon'].values[0]
          lat = current_case['Lat'].values[0]
          lat, lon = jitter((lat, lon)) #jitter
          base_index, to_scene_dist = minHD(lon, lat, heli_arr)

          #Helicopter Available? 
          if(to_scene_dist <= 180): 
            delay_time = np.random.triangular(5,7,10)/60
            prep_time = np.random.triangular(5,7.5,10)/60                         
            calls_dispacted.append(1)

            #Decrement Helicopters Available
            heli_not_returned_baseID.append(base_index)
            heli_arr[base_index] = heli_arr[base_index]-1
        
            cancelDelay = np.random.exponential(1/lambCancelDelay)

            #Cancel Delay?
            if(cancelDelay > (to_scene_dist/speed)): 
              response_fraction.append(1)
              #Time Calculations
              time_on_scene = sc.stats.beta.rvs(a=2.95,b=11072,scale = 1302.6, size =1)[0]
              hospital_index, to_hospital_dist = minHospital(lon, lat)
              #Tracking Stats
              responce_time.append(to_scene_dist/speed)
              time_to_care.append((to_scene_dist/speed) + time_on_scene + (to_hospital_dist/speed) + delay_time + prep_time)
              #Return time
              back_to_base_dist = hosptitalToBaseDist(base_index, hospital_index)
              time_at_hospital = sc.stats.beta.rvs(a=2.90,b=812,scale = 141, size =1)[0]
              heli_return_time.append(overallTime + (to_scene_dist/speed) + time_on_scene + (to_hospital_dist/speed) + time_at_hospital + (back_to_base_dist/speed) + delay_time + prep_time)
          
            else:
              response_fraction.append(0)
              heli_return_time.append(overallTime + (2*cancelDelay) + delay_time + prep_time)
            
          else:
            calls_dispacted.append(0);
            response_fraction.append(0)

        else:
            calls_dispacted.append(0);
            response_fraction.append(0)

  prcnt_calls_dispacted = sum(calls_dispacted)/(len(calls_dispacted)+1)
  avg_rep_time =  sum(responce_time)/(len(responce_time)+1)
  avg_def_car = sum(time_to_care)/(len(time_to_care)+1)
  responce_fraction = sum(response_fraction)/(len(response_fraction)+1)
  utilization = sum(response_fraction)/numSim/amount_of_helis
  rep_time_sd = np.std(responce_time)
  def_car_sd = np.std(time_to_care)
  rep_time_n = len(responce_time) + 1
  def_car_n = len(time_to_care) + 1
  return heli_arr_ORGINAL, prcnt_calls_dispacted, avg_rep_time, avg_def_car, responce_fraction, utilization, rep_time_sd, def_car_sd, rep_time_n, def_car_n



In [ ]:
#____________________ Simulation Function: Number of helicopters and Number of Trials ________________

def simFunctionHeli(amount_of_helis, number_of_trails, number_of_days_per_trial, hosp_with_bases, bases_loc):
  #Value variables
  global_arr = []
  global_percent_dispacted = 0
  global_avg_rep_time = 100
  global_avg_def_care = 0
  global_responce_faction = 0
  global_utilization = 0

  for i in range(number_of_trails):
    arr, val1, val2, val3, val4, val5, val6, val7, val8, val9 = simFunctionDays(number_of_days_per_trial, amount_of_helis, hosp_with_bases, bases_loc)
    if(val2 < global_avg_rep_time):
      global_arr = arr
      global_percent_dispacted = val1
      global_avg_rep_time = val2
      global_avg_def_care = val3
      global_responce_faction = val4
      global_utilization = val5
      g_rep_time_sd = val6
      g_def_car_sd = val7
      g_rep_time_n = val8
      g_def_car_n = val9
  return global_arr, global_percent_dispacted, global_avg_rep_time, global_avg_def_care, global_responce_faction, global_utilization, g_rep_time_sd, g_def_car_sd, g_rep_time_n, g_def_car_n

# 10 Base Simulation Calls

In [ ]:
#____________________ ACTUAL Simulation for ALL hospitals have bases________________
#Simulation Varibles
number_of_trails = 100
number_of_days_per_trial = 20

hosp_with_bases = np.array(['Sarye',
                       'Albany',
                       'Syracuse',
                       'Rochester',
                       'Elmira',
                       'Binghamton',
                       'Ithaca',
                       'Buffalo',
                       'Utica',
                       'Watertown'])
                      
bases_loc = np.array([[41.979, 76.5155], 
                     [42.6526, 73.7562] , 
                     [43.0481, 76.1474] , 
                     [43.1566  , 77.6088] , 
                     [42.0898 , 76.8077] , 
                     [42.0987, 75.918 ], 
                     [42.444 , 76.5019] , 
                     [42.8864 , 78.8784] , 
                     [43.1009 , 75.2327] , 
                     [43.9748 , 75.9108]])

#Number of Helicopters        
for numHelis in range(1,13):
  arr, val1, val2, val3, val4, val5, var6, var7, var8, var9 = simFunctionHeli(numHelis, number_of_trails, number_of_days_per_trial, hosp_with_bases, bases_loc)
  print("Number of helicopters: ", numHelis)
  print("Heli Distribution: ", arr)
  print("Percent Dispatched: ", val1)
  print("Average Response Time: ", val2)
  print("Average Definitive Care Time: ", val3)
  print("Response Fraction: ", val4)
  print("Utilization: ", val5)
  print("Response Time SD: ", var6)
  print("Response Time N: ", var7)
  print("Definitive Care SD: ", var8)
  print("Definitive Care N: ", var9)
  print("_________________________ ")
  print("")

KeyboardInterrupt: ignored

# K-Means Add-On

In [ ]:
|#Random sample
samp = data.iloc[np.random.choice(range(0,data.shape[0]), size = data.shape[0]), 3:5]
kmeans = KMeans(n_clusters=5, random_state=0).fit(np.array(samp))

In [ ]:
kmeans.cluster_centers_

In [ ]:
nearest_hosps = [[42.6526, 73.7562], [42.444, 76.5019], [42.8864, 78.8784], [43.1566, 77.6088], [43.0481, 76.1474]]
hosp_lat, hosp_lon = zip(*nearest_hosps)

plot = plt.scatter(samp.Lon, samp.Lat, alpha=0.1, marker = '.', c = kmeans.labels_)
ax=plt.gca()
ax.invert_xaxis()
plt.xlabel("Longitude")
plt.ylabel("Latitude")
k_lat, k_lon = zip(*kmeans.cluster_centers_)
plt.scatter(k_lon, k_lat, c='red')
plt.scatter(hosp_lon, hosp_lat, c='pink')

In [ ]:
#Rochester, Buffalo, Ithaca, Syracuse, Albany
nearest_hosps = [[42.6526, 73.7562], [42.444, 76.5019], [42.8864, 78.8784], [43.1566, 77.6088], [43.0481, 76.1474]]
hosp_lat, hosp_lon = zip(*nearest_hosps)

h1 = inverse_haversine(nearest_hosps[0],180,Direction.NORTH)[0]-hosp_lat[0]
w1 = inverse_haversine(nearest_hosps[0],180,Direction.WEST)[1]-hosp_lon[0]

h2 = inverse_haversine(nearest_hosps[1],180,Direction.NORTH)[0]-hosp_lat[1]
w2 = inverse_haversine(nearest_hosps[1],180,Direction.WEST)[1]-hosp_lon[1]

h3 = inverse_haversine(nearest_hosps[2],180,Direction.NORTH)[0]-hosp_lat[2]
w3 = inverse_haversine(nearest_hosps[2],180,Direction.WEST)[1]-hosp_lon[2]

h4 = inverse_haversine(nearest_hosps[3],180,Direction.NORTH)[0]-hosp_lat[3]
w4 = inverse_haversine(nearest_hosps[3],180,Direction.WEST)[1]-hosp_lon[3]

h5 = inverse_haversine(nearest_hosps[4],180,Direction.NORTH)[0]-hosp_lat[4]
w5 = inverse_haversine(nearest_hosps[4],180,Direction.WEST)[1]-hosp_lon[4]

plot = plt.scatter(samp.Lon, samp.Lat, alpha=0.1, marker = '.', c = kmeans.labels_)
ax=plt.gca()
ax.invert_xaxis()
plt.xlabel("Longitude")
plt.ylabel("Latitude")
k_lat, k_lon = zip(*kmeans.cluster_centers_)
plt.scatter(k_lon, k_lat, c='red')
plt.scatter(hosp_lon, hosp_lat, c='pink')

e1 = Ellipse(xy=(73.7562, 42.6526), width=w1, height=h1, 
                        edgecolor='purple', fc='None', lw=2)
e2 = Ellipse(xy=(76.5019, 42.444), width=w2, height=h2, 
                        edgecolor='yellow', fc='None', lw=2)
e3 = Ellipse(xy=(78.8784, 42.8864), width=w3, height=h3, 
                        edgecolor='green', fc='None', lw=2)
e4 = Ellipse(xy=(77.6088, 43.1566), width=w4, height=h4, 
                        edgecolor='navy', fc='None', lw=2)
e5 = Ellipse(xy=(76.1474, 43.0481), width=w5, height=h5, 
                        edgecolor='teal', fc='None', lw=2)
ax.add_patch(e1)
ax.add_patch(e2)
ax.add_patch(e3)
ax.add_patch(e4)
ax.add_patch(e5)

In [ ]:
#Move from Syracuse to Utica (?)
new_hosps = [[42.6526, 73.7562], [42.444, 76.5019], [42.8864, 78.8784], [43.1566, 77.6088], [43.1009 , 75.2327]]
hosp_lat, hosp_lon = zip(*new_hosps)

h1 = inverse_haversine(new_hosps[0],180,Direction.NORTH)[0]-hosp_lat[0]
w1 = inverse_haversine(new_hosps[0],180,Direction.WEST)[1]-hosp_lon[0]

h2 = inverse_haversine(new_hosps[1],180,Direction.NORTH)[0]-hosp_lat[1]
w2 = inverse_haversine(new_hosps[1],180,Direction.WEST)[1]-hosp_lon[1]

h3 = inverse_haversine(new_hosps[2],180,Direction.NORTH)[0]-hosp_lat[2]
w3 = inverse_haversine(new_hosps[2],180,Direction.WEST)[1]-hosp_lon[2]

h4 = inverse_haversine(new_hosps[3],180,Direction.NORTH)[0]-hosp_lat[3]
w4 = inverse_haversine(new_hosps[3],180,Direction.WEST)[1]-hosp_lon[3]

h5 = inverse_haversine(new_hosps[4],180,Direction.NORTH)[0]-hosp_lat[4]
w5 = inverse_haversine(new_hosps[4],180,Direction.WEST)[1]-hosp_lon[4]

plot = plt.scatter(samp.Lon, samp.Lat, alpha=0.1, marker = '.', c = kmeans.labels_)
ax=plt.gca()
ax.invert_xaxis()
plt.xlabel("Longitude")
plt.ylabel("Latitude")
k_lat, k_lon = zip(*kmeans.cluster_centers_)
plt.scatter(k_lon, k_lat, c='red')
plt.scatter(hosp_lon, hosp_lat, c='pink')

e1 = Ellipse(xy=(73.7562, 42.6526), width=w1, height=h1, 
                        edgecolor='purple', fc='None', lw=2)
e2 = Ellipse(xy=(76.5019, 42.444), width=w2, height=h2, 
                        edgecolor='yellow', fc='None', lw=2)
e3 = Ellipse(xy=(78.8784, 42.8864), width=w3, height=h3, 
                        edgecolor='green', fc='None', lw=2)
e4 = Ellipse(xy=(77.6088, 43.1566), width=w4, height=h4, 
                        edgecolor='navy', fc='None', lw=2)
e5 = Ellipse(xy=(75.2327, 43.1009), width=w5, height=h5, 
                        edgecolor='teal', fc='None', lw=2)
ax.add_patch(e1)
ax.add_patch(e2)
ax.add_patch(e3)
ax.add_patch(e4)
ax.add_patch(e5)

In [ ]:
#Move Utica to Watertown
new_hosps = [[42.6526, 73.7562], [42.444, 76.5019], [42.8864, 78.8784], [43.1566, 77.6088], [43.9748 , 75.9108]]
hosp_lat, hosp_lon = zip(*new_hosps)

h1 = inverse_haversine(new_hosps[0],180,Direction.NORTH)[0]-hosp_lat[0]
w1 = inverse_haversine(new_hosps[0],180,Direction.WEST)[1]-hosp_lon[0]

h2 = inverse_haversine(new_hosps[1],180,Direction.NORTH)[0]-hosp_lat[1]
w2 = inverse_haversine(new_hosps[1],180,Direction.WEST)[1]-hosp_lon[1]

h3 = inverse_haversine(new_hosps[2],180,Direction.NORTH)[0]-hosp_lat[2]
w3 = inverse_haversine(new_hosps[2],180,Direction.WEST)[1]-hosp_lon[2]

h4 = inverse_haversine(new_hosps[3],180,Direction.NORTH)[0]-hosp_lat[3]
w4 = inverse_haversine(new_hosps[3],180,Direction.WEST)[1]-hosp_lon[3]

h5 = inverse_haversine(new_hosps[4],180,Direction.NORTH)[0]-hosp_lat[4]
w5 = inverse_haversine(new_hosps[4],180,Direction.WEST)[1]-hosp_lon[4]

plot = plt.scatter(samp.Lon, samp.Lat, alpha=0.1, marker = '.', c = kmeans.labels_)
ax=plt.gca()
ax.invert_xaxis()
plt.xlabel("Longitude")
plt.ylabel("Latitude")
k_lat, k_lon = zip(*kmeans.cluster_centers_)
plt.scatter(k_lon, k_lat, c='red')
plt.scatter(hosp_lon, hosp_lat, c='pink')

e1 = Ellipse(xy=(73.7562, 42.6526), width=w1, height=h1, 
                        edgecolor='purple', fc='None', lw=2)
e2 = Ellipse(xy=(76.5019, 42.444), width=w2, height=h2, 
                        edgecolor='yellow', fc='None', lw=2)
e3 = Ellipse(xy=(78.8784, 42.8864), width=w3, height=h3, 
                        edgecolor='green', fc='None', lw=2)
e4 = Ellipse(xy=(77.6088, 43.1566), width=w4, height=h4, 
                        edgecolor='navy', fc='None', lw=2)
e5 = Ellipse(xy=(75.9108, 43.9748), width=w5, height=h5, 
                        edgecolor='teal', fc='None', lw=2)
ax.add_patch(e1)
ax.add_patch(e2)
ax.add_patch(e3)
ax.add_patch(e4)
ax.add_patch(e5)

In [ ]:
#1 pad
samp = data.iloc[np.random.choice(range(0,data.shape[0]), size = data.shape[0]), 3:5]
kmeans = KMeans(n_clusters=1).fit(np.array(samp))

plot = plt.scatter(samp.Lon, samp.Lat, alpha=0.1, marker = '.', c = kmeans.labels_)
ax=plt.gca()
ax.invert_xaxis()
plt.xlabel("Longitude")
plt.ylabel("Latitude")
k_lat, k_lon = zip(*kmeans.cluster_centers_)
plt.scatter(k_lon, k_lat, c='red')

In [ ]:
#2 pads
samp = data.iloc[np.random.choice(range(0,data.shape[0]), size = data.shape[0]), 3:5]
kmeans = KMeans(n_clusters=2).fit(np.array(samp))

plot = plt.scatter(samp.Lon, samp.Lat, alpha=0.1, marker = '.', c = kmeans.labels_)
ax=plt.gca()
ax.invert_xaxis()
plt.xlabel("Longitude")
plt.ylabel("Latitude")
k_lat, k_lon = zip(*kmeans.cluster_centers_)
plt.scatter(k_lon, k_lat, c='red')

In [ ]:
#3 pads
samp = data.iloc[np.random.choice(range(0,data.shape[0]), size = data.shape[0]), 3:5]
kmeans = KMeans(n_clusters=3).fit(np.array(samp))

plot = plt.scatter(samp.Lon, samp.Lat, alpha=0.1, marker = '.', c = kmeans.labels_)
ax=plt.gca()
ax.invert_xaxis()
plt.xlabel("Longitude")
plt.ylabel("Latitude")
k_lat, k_lon = zip(*kmeans.cluster_centers_)
plt.scatter(k_lon, k_lat, c='red')

In [ ]:
#4 pads
samp = data.iloc[np.random.choice(range(0,data.shape[0]), size = data.shape[0]), 3:5]
kmeans = KMeans(n_clusters=4).fit(np.array(samp))

plot = plt.scatter(samp.Lon, samp.Lat, alpha=0.1, marker = '.', c = kmeans.labels_)
ax=plt.gca()
ax.invert_xaxis()
plt.xlabel("Longitude")
plt.ylabel("Latitude")
k_lat, k_lon = zip(*kmeans.cluster_centers_)
plt.scatter(k_lon, k_lat, c='red')

In [ ]:
#"Bootstrap"
#THIS TAKES LIKE 7 MINUTES TO RUN
centers = []

for i in range(1000):
    samp = data.iloc[np.random.choice(range(0,data.shape[0]), size = data.shape[0]), 3:5]
    kmeans = KMeans(n_clusters=5).fit(np.array(samp))
    centers.append(kmeans.cluster_centers_)

In [ ]:
nearest_hosps = [[42.6526, 73.7562], [42.444, 76.5019], [42.8864, 78.8784], [43.1566, 77.6088], [43.0481, 76.1474]]
hosp_lat, hosp_lon = zip(*nearest_hosps)

plot = plt.scatter(samp.Lon, samp.Lat, alpha=0.1, marker = '.', c = kmeans.labels_)
ax=plt.gca()
ax.invert_xaxis()
plt.xlabel("Longitude")
plt.ylabel("Latitude")
k_lat, k_lon = zip(*np.resize(centers, (5000,2)))
plt.scatter(k_lon, k_lat, c='red', alpha = 0.1)
plt.scatter(hosp_lon, hosp_lat, c='pink')

# 5 base Simulation Calls

In [ ]:
#____________________ ACTUAL Simulation for 5 hospitals have bases________________
#Simulation Varibles
number_of_trails = 100
number_of_days_per_trial = 30

hosp_with_bases = np.array([
                       'Albany', 
                       'Rochester', 
                       'Utica', 
                       'Ithaca', 
                       'Buffalo'])
                      
bases_loc = np.array([
                     [43.0481, 76.1474] , 
                     [43.1566  , 77.6088] , 
                     [43.1009 , 75.2327] , 
                     [42.444 , 76.5019] , 
                     [42.8864 , 78.8784]])

#Number of Helicopters        
for numHelis in range(1,13):
  arr, val1, val2, val3, val4, val5 = simFunctionHeli(numHelis, number_of_trails, number_of_days_per_trial, hosp_with_bases, bases_loc)
  print("Number of helicopters: ", numHelis)
  print("Heli Distribution: ", arr)
  print("Percent Dispatched: ", val1)
  print("Average Response Time: ", val2)
  print("Average Definitive Care Time: ", val3)
  print("Response Fraction: ", val4)
  print("Utilization: ", val5)
  print("_________________________ ")
  print("")

In [ ]:
#____________________ ACTUAL Simulation for 5 hospitals have bases________________
#But with Watertown instead of Utica
#Simulation Varibles
number_of_trails = 200
number_of_days_per_trial = 3

hosp_with_bases = np.array([
                       'Albany', 
                       'Rochester', 
                       'Watertown', 
                       'Ithaca', 
                       'Buffalo'])
                      
bases_loc = np.array([
                     [43.0481, 76.1474] , 
                     [43.1566  , 77.6088] , 
                     [43.9748 , 75.9108] , 
                     [42.444 , 76.5019] , 
                     [42.8864 , 78.8784]])

#Number of Helicopters        
for numHelis in range(1,13):
  arr, val1, val2, val3, val4, val5 = simFunctionHeli(numHelis, number_of_trails, number_of_days_per_trial, hosp_with_bases, bases_loc)
  print("Number of helicopters: ", numHelis)
  print("Heli Distribution: ", arr)
  print("Percent Dispatched: ", val1)
  print("Average Response Time: ", val2)
  print("Average Definitive Care Time: ", val3)
  print("Response Fraction: ", val4)
  print("Utilization: ", val5)
  print("_________________________ ")
  print("")

KeyboardInterrupt: ignored